In [2]:
import osmnx as ox
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
from random import choices

# define class for building
class Building:
    def __init__(self, id, geometry, cen_pt, bld_type):
        self.id = id
        self.geometry = geometry
        self.cen_pt = cen_pt
        self.bld_type = bld_type

# importing osm location
address = "Çimentepe Mahallesi"
buildings = ox.features.features_from_address(address, tags={'building': True}, dist=350)

# creating object for each building
bld_lst = []
for i in buildings.index:
    id = i[1]
    geo = buildings.loc[i, 'geometry']
    cen_pt = geo.centroid
    bld_type = buildings.loc[i, 'building']

    bld = Building(id, geo, cen_pt, bld_type)
    bld_lst.append(bld)

In [1]:
import numpy as np
from shapely.geometry import Polygon
from matplotlib.patches import Polygon as mpl_polygon

cen_pts = gpd.GeoDataFrame({'points': [o.cen_pt for o in bld_lst]}, geometry = 'points')

# set bounds of graph to the bounds of center points
xmin, ymin, xmax, ymax = cen_pts.total_bounds
x_range = xmax - xmin
y_range = ymax - ymin

# set stepsize
width = x_range / 10
height = y_range / 10

# create cols and rows based on bounds and stepsize
cols = list(np.arange(xmin, xmax + width, width))
rows = list(np.arange(ymin, ymax + height, height))

# create empty graph
fig, ax = plt.subplots()

sorted_blds = sorted(bld_lst, key=lambda x: (x.cen_pt.y, x.cen_pt.x))
print([[o.cen_pt.y, o.cen_pt.x] for o in sorted_blds])

# create cells and plot to graph, add buildings to subarea classes
polygons = []
i = 0
for x in cols[:-1]:
    # while the buildings have x values in the range of this column, add them to a list
    # filt_blds = []
    # while (sorted_blds[i].cen_pt.x >= x) and (sorted_blds[i].cen_pt.x < x+width): 
    #     filt_blds.append(sorted_blds[i])
    #     # print(filt_blds)
    for y in rows[:-1]:
        polygon = Polygon([(x,y), (x+width, y), (x+width, y+height), (x, y+height)])
        polygons.append(polygon)
        ax.plot(*polygon.exterior.xy)



# this is plotting the geometry of the buildings
# save each geometry attribute in a list
geos = [o.geometry for o in bld_lst]

# choices of different colours with corresponding weights
population = ['green', 'blue', 'yellow', 'orange', 'red']
weights = [0.45, 0.05, 0.15, 0.12, 0.23]

# for every geometry choose a colour, fill the geometry and plot to graph
for g in geos:
    c = choices(population, weights)[0]
    x, y = g.exterior.xy
    ax.fill(x, y, color = c)

# Display the plot
plt.show()

NameError: name 'gpd' is not defined

In [46]:
import numpy as np
from shapely.geometry import Polygon
from matplotlib.patches import Polygon as mpl_polygon

cen_pts = gpd.GeoDataFrame({'points': [o.cen_pt for o in bld_lst]}, geometry = 'points')

xmin, ymin, xmax, ymax = cen_pts.total_bounds
x_range = xmax - xmin
y_range = ymax - ymin

length = x_range / 10
width = y_range / 10

cols = list(np.arange(xmin, xmax + width, width))
rows = list(np.arange(ymin, ymax + length, length))

print(len(cols))
print(len(rows))
print(len(cols) * len(rows))
# create cells and plot to graph
# fig, ax = plt.subplots()
polygons = []
for x in cols[:-1]:
    for y in rows[:-1]:
        polygon = Polygon([(x,y), (x+wide, y), (x+wide, y+length), (x, y+length)])
        polygons.append(polygon)
        # mpl_poly = mpl_polygon(list(polygon.exterior.coords), edgecolor='r', facecolor='none')
        # ax.add_patch(mpl_poly)
        ax.plot(*polygon.exterior.xy)
print(len(polygons))        
# plt.show()

15
9
135
112


In [28]:
"""Hops flask middleware example"""
import sys
import os
from flask import Flask
from PIL import Image
import rhino3dm as rs

# load ghhops-server-py source from this directory
import ghhops_server as hs

# register hops app as middleware
app = Flask(__name__)
hops: hs.HopsFlask = hs.Hops(app)

# flask app can be used for other stuff directly
@app.route("/help")
def help():
    return "Welcome to Grashopper Hops for CPython!"


@app.route("/update", methods=["POST"])
def update():
    return "Update example!"

# component
@hops.component(
    "/imft",
    name="ImportFootprints",
    nickname="IF",
    description="Import Building Footprints",
    outputs=[hs.HopsPoint("Footprints", "F", "List of footprints imported")]
)

def import_footprints():
    c_pts = []
    for geo in geos:
        points = []
        for point in geo.exterior.coords:
            x, y = point
            rh_point = rs.Point3d(x, y, 0)
            points.append(rh_point)
        c_pts.append(points)
    polygon = rs.Curve.CreateControlPointCurve(c_pts)
    return hs.HopsCurve(polygon)

if __name__ == "__main__":
    app.run()


 * Serving Flask app "__main__" (lazy loading)


 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


[INFO]  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[INFO] 127.0.0.1 - - [12/Oct/2023 14:08:47] "GET /imft HTTP/1.1" 200 -
[INFO] 127.0.0.1 - - [12/Oct/2023 14:08:47] "GET /imft HTTP/1.1" 200 -
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x1ddd0a31040>
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x1ddd0a317c0>
[INFO] Solving using legacy API: <HopsComponent /imft [ -> ImportFootprints -> Footprints] >
[INFO] 127.0.0.1 - - [12/Oct/2023 14:08:47] "POST /solve HTTP/1.1" 200 -
